In [ ]:
import cv2 # OpenCV
import numpy as np
import skimage.measure
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from scipy.ndimage import uniform_filter
from scipy import stats
import os
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
# cv2.__version__

In [ ]:
#função de Informação Espacial

def Inf_spatial(img_grayscale):
  sh = cv2.Sobel(img_grayscale, cv2.CV_64F, 1, 0, ksize = 1)
  sv = cv2.Sobel(img_grayscale, cv2.CV_64F, 0, 1, ksize = 1)

  #SIr = np.sqrt (sh **2 + sv **2)
  SIr = np.sqrt(np.square(sh) + np.square(sv))

  SI_mean = np.sum(SIr) / (SIr.shape[0] * SIr.shape[1])
  SI_rms = np.sqrt(np.sum(SIr ** 2) / (SIr.shape[0] * SIr.shape[1]))
  SI_stdev = np.sqrt(np.sum(SIr ** 2 - SI_mean ** 2) / (SIr.shape[0] * SIr.shape[1]))

  return SI_stdev

In [ ]:
def resize_down(input_image):

  # Get the dimensions of the source image
  j, k = input_image.shape

  # Specify the new image dimensions for downsampling
  x_new = j // 2
  y_new = k // 2

  # Determine the scaling ratios
  x_scale = j / x_new
  y_scale = k / y_new

  # Initialize an output image buffer
  output_image = np.zeros((x_new, y_new), dtype=np.uint8)

  # Generate the output image
  for count1 in range(x_new):
      for count2 in range(y_new):
          output_image[count1, count2] = input_image[int(count1 * x_scale), int(count2 * y_scale)]

  return output_image


In [ ]:
def resize_up(input_image):

  # Get the dimensions of the source image
  j, k = input_image.shape

  # Determine how much larger we want the new image (should be an integer)
  scale = 2

  # Initialize an output image buffer
  output_image = np.zeros((scale * j, scale * k), dtype=np.uint8)

  # Generate the output image
  for count1 in range(0, scale * j, scale):
      for count2 in range(0, scale * k, scale):
          for count3 in range(scale):
              for count4 in range(scale):
                  output_image[count1 + count3, count2 + count4] = input_image[count1 // scale, count2 // scale]

  return output_image


In [ ]:
def median_filter(input_image, window_width, window_height):
    image_width, image_height = input_image.shape
    edgex = window_width // 2
    edgey = window_height // 2

    output_image = np.zeros_like(input_image, dtype=np.uint8)

    for x in range(edgex, image_width - edgex):
        for y in range(edgey, image_height - edgey):
            i = 0
            window = np.empty(window_width * window_height, dtype=np.uint8)
            for fx in range(window_width):
                for fy in range(window_height):
                    window[i] = input_image[x + fx - edgex][y + fy - edgey]
                    i += 1
            window.sort()
            output_image[x][y] = window[window_width * window_height // 2]

    return output_image


In [ ]:
def mean_filter(input_image, window_width, window_height):
    # Create a uniform filter kernel with the specified window size
    kernel = np.ones((window_width, window_height)) / (window_width * window_height)

    # Apply the mean filter using convolution
    output_image = uniform_filter(input_image, size=(window_width, window_height))

    return output_image.astype(np.uint8)


In [ ]:
def mode_filter(input_image, window_width, window_height):
    image_width, image_height = input_image.shape
    edgex = window_width // 2
    edgey = window_height // 2

    output_image = np.zeros_like(input_image, dtype=np.uint8)

    for x in range(edgex, image_width - edgex):
        for y in range(edgey, image_height - edgey):
            i = 0
            window = np.empty(window_width * window_height, dtype=np.uint8)
            for fx in range(window_width):
                for fy in range(window_height):
                    window[i] = input_image[x + fx - edgex][y + fy - edgey]
                    i += 1
            mode_value = stats.mode(window).mode[0]
            output_image[x][y] = mode_value

    return output_image


In [ ]:
imagens_originais_dir = '/content/drive/MyDrive/PDI/projeto01/imagens/originais/'
imagens_originais_reduzidas_dir = '/content/drive/MyDrive/PDI/projeto01/imagens/reduzidas_fator_2/'
resultados_dir = '/content/drive/MyDrive/PDI/projeto01/resultados/'
imagens_resultantes_downsampling_dir = '/content/drive/MyDrive/PDI/projeto01/imagens/resultantes_downsampling/'
imagens_resultantes_upsampling_dir = '/content/drive/MyDrive/PDI/projeto01/imagens/resultantes_upsampling/'

In [ ]:
# Redução da Imagem - Downsampling

for arquivo in os.listdir(imagens_originais_dir):

  # Carregar imagem original, entrada para o processamento
  imagem_original = os.path.join(imagens_originais_dir, arquivo)
  imagem_entrada = cv2.imread(imagem_original, cv2.IMREAD_GRAYSCALE)

  # Carregar imagem original reduzida para cálculo das métricas pós processamento
  imagem_original_reduzida = os.path.join(imagens_originais_reduzidas_dir, arquivo)
  imagem_metrica = cv2.imread(imagem_original_reduzida, cv2.IMREAD_GRAYSCALE)

  # Resize the input image
  imagem_redimensionada = resize_down(imagem_entrada)

  # Resultados para o arquivo/imagem. Resetados a cada iteração de arquivo
  resultados_arquivo_psnr = []
  resultados_arquivo_ssim = []

  # Definição das janelas de filtragem
  tamanho_janelas = [2, 3, 4, 5]

  # Aplicação dos filtros para cada janela
  for tamanho in tamanho_janelas:

    largura = tamanho
    altura = tamanho

    # Filtros e gravação dos arquivos de saída
    imagem_saida1 = median_filter(imagem_redimensionada, largura, altura)
    cv2.imwrite(f'{imagens_resultantes_downsampling_dir}{arquivo}_mediana_{largura}x{altura}.jpg', imagem_saida1)

    imagem_saida2 = mean_filter(imagem_redimensionada, largura, altura)
    cv2.imwrite(f'{imagens_resultantes_downsampling_dir}{arquivo}_media_{largura}x{altura}.jpg', imagem_saida2)

    imagem_saida3 = mode_filter(imagem_redimensionada, largura, altura)
    cv2.imwrite(f'{imagens_resultantes_downsampling_dir}{arquivo}_moda_{largura}x{altura}.jpg', imagem_saida3)

    # Métricas e acúmulo dos resultados para o arquivo/imagem
    # PSNR
    resultados_arquivo_psnr.append(psnr(imagem_metrica, imagem_saida1))
    resultados_arquivo_psnr.append(psnr(imagem_metrica, imagem_saida2))
    resultados_arquivo_psnr.append(psnr(imagem_metrica, imagem_saida3))

    # SSIM
    resultados_arquivo_ssim.append(ssim(imagem_metrica,
                                        imagem_saida1,
                                        data_range=imagem_saida1.max() - imagem_saida1.min()))
    resultados_arquivo_ssim.append(ssim(imagem_metrica,
                                        imagem_saida2,
                                        data_range=imagem_saida2.max() - imagem_saida2.min()))
    resultados_arquivo_ssim.append(ssim(imagem_metrica,
                                        imagem_saida3,
                                        data_range=imagem_saida3.max() - imagem_saida3.min()))

    # --------------- Fim do for

  # Reduzindo e filtrando através dos métodos do OpenCV e gravação dos arquivos de saída
  # Usado fator redução por 5, alinhado com a função resize
  imagem_saida4 = cv2.resize(imagem_entrada, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_NEAREST)
  cv2.imwrite(f'{imagens_resultantes_downsampling_dir}{arquivo}_cv2INTER_NEAREST.jpg', imagem_saida4)

  imagem_saida5 = cv2.resize(imagem_entrada, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_CUBIC)
  cv2.imwrite(f'{imagens_resultantes_downsampling_dir}{arquivo}_cv2INTER_CUBIC.jpg', imagem_saida5)

  imagem_saida6 = cv2.resize(imagem_entrada, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
  cv2.imwrite(f'{imagens_resultantes_downsampling_dir}{arquivo}_cv2INTER_AREA.jpg', imagem_saida6)

  # Métricas para saídas pelo OpenCV e acúmulo dos resultados para o arquivo/imagem
  # PSNR
  resultados_arquivo_psnr.append(psnr(imagem_metrica, imagem_saida4))
  resultados_arquivo_psnr.append(psnr(imagem_metrica, imagem_saida5))
  resultados_arquivo_psnr.append(psnr(imagem_metrica, imagem_saida6))

  # SSIM
  resultados_arquivo_ssim.append(ssim(imagem_metrica,
                                      imagem_saida4,
                                      data_range=imagem_saida4.max() - imagem_saida4.min()))
  resultados_arquivo_ssim.append(ssim(imagem_metrica,
                                      imagem_saida5,
                                      data_range=imagem_saida5.max() - imagem_saida5.min()))
  resultados_arquivo_ssim.append(ssim(imagem_metrica,
                                      imagem_saida6,
                                      data_range=imagem_saida6.max() - imagem_saida6.min()))

  np.savetxt(rf'{resultados_dir}downsampling_{arquivo}_psnr.txt',
             resultados_arquivo_psnr,
             fmt='%.2f',
             delimiter=',',
             newline='\n')
  np.savetxt(rf'{resultados_dir}downsampling_{arquivo}_ssim.txt',
             resultados_arquivo_ssim,
             fmt='%.2f',
             delimiter=',',
             newline='\n')



<ipython-input-8-039d8f79a404>:16: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode_value = stats.mode(window).mode[0]


In [ ]:
# Expansão da Imagem - Upsampling

for arquivo in os.listdir(imagens_originais_reduzidas_dir):

  # Carregar imagem original reduzida, entrada para o processamento
  # Carregar imagem original reduzida para cálculo das métricas pós processamento
  imagem_original_reduzida = os.path.join(imagens_originais_reduzidas_dir, arquivo)
  imagem_entrada = cv2.imread(imagem_original_reduzida, cv2.IMREAD_GRAYSCALE)

  # Carregar imagem original para cálculo das métricas pós processamento
  imagem_original = os.path.join(imagens_originais_dir, arquivo)
  imagem_metrica = cv2.imread(imagem_original, cv2.IMREAD_GRAYSCALE)


  # Resize the input image
  imagem_redimensionada = resize_up(imagem_entrada)

  # Resultados para o arquivo/imagem. Resetados a cada iteração de arquivo
  resultados_arquivo_psnr = []
  resultados_arquivo_ssim = []

  # Definição das janelas de filtragem
  tamanho_janelas = [2, 3, 4, 5]

  # Aplicação dos filtros para cada janela
  for tamanho in tamanho_janelas:

    largura = tamanho
    altura = tamanho

    # Filtros e gravação dos arquivos de saída
    imagem_saida1 = median_filter(imagem_redimensionada, largura, altura)
    cv2.imwrite(f'{imagens_resultantes_upsampling_dir}{arquivo}_mediana_{largura}x{altura}.jpg', imagem_saida1)

    imagem_saida2 = mean_filter(imagem_redimensionada, largura, altura)
    cv2.imwrite(f'{imagens_resultantes_upsampling_dir}{arquivo}_media_{largura}x{altura}.jpg', imagem_saida2)

    imagem_saida3 = mode_filter(imagem_redimensionada, largura, altura)
    cv2.imwrite(f'{imagens_resultantes_upsampling_dir}{arquivo}_moda_{largura}x{altura}.jpg', imagem_saida3)

    # Métricas e acúmulo dos resultados para o arquivo/imagem
    # PSNR
    resultados_arquivo_psnr.append(psnr(imagem_metrica, imagem_saida1))
    resultados_arquivo_psnr.append(psnr(imagem_metrica, imagem_saida2))
    resultados_arquivo_psnr.append(psnr(imagem_metrica, imagem_saida3))

    # SSIM
    resultados_arquivo_ssim.append(ssim(imagem_metrica,
                                        imagem_saida1,
                                        data_range=imagem_saida1.max() - imagem_saida1.min()))
    resultados_arquivo_ssim.append(ssim(imagem_metrica,
                                        imagem_saida2,
                                        data_range=imagem_saida2.max() - imagem_saida2.min()))
    resultados_arquivo_ssim.append(ssim(imagem_metrica,
                                        imagem_saida3,
                                        data_range=imagem_saida3.max() - imagem_saida3.min()))

    # --------------- Fim do for

  # Expandindo e filtrando através dos métodos do OpenCV e gravação dos arquivos de saída
  # Usado fator de aumento por 5, alinhado com a função resize
  imagem_saida4 = cv2.resize(imagem_entrada, None, fx=2, fy=2, interpolation=cv2.INTER_NEAREST)
  cv2.imwrite(f'{imagens_resultantes_upsampling_dir}{arquivo}_cv2INTER_NEAREST.jpg', imagem_saida4)

  imagem_saida5 = cv2.resize(imagem_entrada, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
  cv2.imwrite(f'{imagens_resultantes_upsampling_dir}{arquivo}_cv2INTER_CUBIC.jpg', imagem_saida5)

  imagem_saida6 = cv2.resize(imagem_entrada, None, fx=2, fy=2, interpolation=cv2.INTER_AREA)
  cv2.imwrite(f'{imagens_resultantes_upsampling_dir}{arquivo}_cv2INTER_AREA.jpg', imagem_saida6)

  # Métricas para saídas pelo OpenCV e acúmulo dos resultados para o arquivo/imagem
  # PSNR
  resultados_arquivo_psnr.append(psnr(imagem_metrica, imagem_saida4))
  resultados_arquivo_psnr.append(psnr(imagem_metrica, imagem_saida5))
  resultados_arquivo_psnr.append(psnr(imagem_metrica, imagem_saida6))

  # SSIM
  resultados_arquivo_ssim.append(ssim(imagem_metrica,
                                      imagem_saida4,
                                      data_range=imagem_saida4.max() - imagem_saida4.min()))
  resultados_arquivo_ssim.append(ssim(imagem_metrica,
                                      imagem_saida5,
                                      data_range=imagem_saida5.max() - imagem_saida5.min()))
  resultados_arquivo_ssim.append(ssim(imagem_metrica,
                                      imagem_saida6,
                                      data_range=imagem_saida6.max() - imagem_saida6.min()))

  np.savetxt(rf'{resultados_dir}upsampling_{arquivo}_psnr.txt',
             resultados_arquivo_psnr,
             fmt='%.2f',
             delimiter=',',
             newline='\n')
  np.savetxt(rf'{resultados_dir}upsampling_{arquivo}_ssim.txt',
             resultados_arquivo_ssim,
             fmt='%.2f',
             delimiter=',',
             newline='\n')



<ipython-input-8-039d8f79a404>:16: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode_value = stats.mode(window).mode[0]
